## Portfolio Optimization Tools in Python

##### Author: Zachary Wright, CFA, FRM | Last Updated: 01/27/25

Welcome! This is a portfolio project dedicated toward showcasing skills acquired in Python in a quantitative finance scenario and may be updated as of time of viewing.

**Goal:** optimize a portfolio using various methods; maximize Sharpe ratio, minimize variance, and target return and risk.

Tools used:
- yfinance API to gather asset level data dynamically
- Scipy library to optimize asset weightings
- Python class constructor and object-oriented programming techniques for efficiency and modularity

First I will install and then import the required libraries:

In [1]:
!pip install pandas
!pip install numpy
!pip install yfinance
!pip install scipy

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.5 MB 12.2 MB/s eta 0:00:01
   ----------------- ---------------------- 5.0/11.5 MB 12.6 MB/s eta 0:00:01
   --------------------------- ------------ 7.9/11.5 MB 13.5 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 14.4 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 14.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------- ------------------------------ 2.9/12.6 MB 15.2 MB/s eta 0:00:01
   -------------- ------------------------- 4.7/12.6 MB 12.4 MB/s eta 0:00:01
   -------------------- ------------------- 6.6/12.6 MB 10.9 MB/s eta 0:00:01
   ------------------------- -------------- 8.1/12.6 MB 10.1 MB/s eta 0:00:01
   ----------------------------- ---------- 9.4/12.6 MB 9.8 MB/s eta 0:00:01



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Zack\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Zack\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     -------------------------------------- 948.2/948.2 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.8 MB 5.6 MB/s eta 0:00:01
   ------------------------ --------------- 2.4/3.8 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 6.9 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=139064 sha256=0022c738a7b39b6709c8f74c2f0c6fc89a8b8eaee046210624da61ee3ea25502
  Stored in directory: c:\users\zack\appdata\local\pip\cache\wheels\


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Zack\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Zack\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
    --------------------------------------- 1.0/43.6 MB 25.4 MB/s eta 0:00:02
   -- ------------------------------------- 3.1/43.6 MB 13.2 MB/s eta 0:00:04
   ----- ---------------------------------- 5.8/43.6 MB 12.2 MB/s eta 0:00:04
   ------- -------------------------------- 8.4/43.6 MB 12.4 MB/s eta 0:00:03
   ---------- ----------------------------- 11.3/43.6 MB 12.8 MB/s eta 0:00:03
   ------------ --------------------------- 13.6/43.6 MB 12.8 MB/s eta 0:00:03
   -------------- ------------------------- 15.7/43.6 MB 11.9 MB/s eta 0:00:03
   --------------- ------------------------ 17.0/43.6 MB 11.3 MB/s eta 0:00:03
   ---------------- ----------------------- 18.4/43.6 MB 10.6 MB/s eta 0:00:03
   ------------------ --------------------- 20.2/43.6 MB 10.3 MB/s eta 0:00:03
   ------------------- -------------------- 21.8/43.6 MB 10.0 MB/s eta 0:0

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

Next, since Python is an interpreted language, I will define a Portfolio class and its associated functions and methods first.

In [7]:
#Create class portfolio - we can theoretically have multiple portfolios with different asset and return data series.
class Portfolio:
    def __init__(self, returns, weights):
        self.returns = returns
        self.weights = weights
    
    #Return the return of portfolio
    def get_return(self, weights=None):
        avg_asset_returns = self.returns.mean()
        if weights is None:
            weights = self.weights
        port_return = sum(w * avg_ret for w, avg_ret in zip(weights, avg_asset_returns))
        return port_return

    #Return the covariance of the portfolio
    def get_cov(self):
        return self.returns.cov()

    #Return the variance of the portfolio
    def get_var(self, weights=None):
        if weights is None:
            weights = self.weights
        cov_matrix = self.get_cov()
        weights = np.array(weights)
        port_variance = weights.T @ cov_matrix @ weights
        return port_variance

    #Get the return series data of provided tickers from yfinance API
    def get_returns_from_yfinance(self, tickers, start_date, end_date):
        all_data = {}
        for ticker in tickers:
            data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
            returns = data["Adj Close"].pct_change().dropna()
            all_data[ticker] = returns
        return pd.DataFrame(all_data)

    #Optimization function: can maximize Sharpe Ratio, minimize variance, and target return and risk
    def optimize(self, objective="sharpe", target_return=None, target_risk=None):
        n_assets = len(self.weights)
        avg_returns = self.returns.mean()
        cov_matrix = self.get_cov()

        #Risk-free rate proxy from TBLL ETF yield through yfinance
        tbll = yf.Ticker("TBLL")
        rfr_proxy = tbll.info.get('dividendYield', 0)

        #Define constraints
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(n_assets)]

        #Define the objective function
        if objective == "sharpe":
            def objective_function(weights):
                port_return = self.get_return(weights)
                port_variance = self.get_var(weights)
                sharpe_ratio = (port_return - rfr_proxy) / np.sqrt(port_variance)
                return -sharpe_ratio
        elif objective == "min_variance":
            def objective_function(weights):
                return self.get_var(weights)
        elif objective == "target_return":
            if target_return is None:
                raise ValueError("You must specify a target_return for this objective.")
            def objective_function(weights):
                return self.get_var(weights)
            constraints.append({'type': 'eq', 'fun': lambda w: self.get_return(w) - target_return})
        elif objective == "target_risk":
            if target_risk is None:
                raise ValueError("You must specify a target_risk for this objective.")
            def objective_function(weights):
                return abs(self.get_var(weights) - target_risk)
        else:
            raise ValueError("Invalid optimization objective.")

        #Optimize
        result = minimize(objective_function, self.weights, constraints=constraints, bounds=bounds)
        if result.success:
            self.weights = result.x
        return result

From here, we can import our portfolio's assets and their associated monthly returns. 

In [4]:
#Create a dataframe with asset and return data
tickers = ["IVV", "IXUS", "IEMG"]
data = {
    "Asset_1": [0.02, 0.03, 0.01, 0.04],
    "Asset_2": [0.01, 0.02, 0.00, 0.03],
    "Asset_3": [0.03, 0.01, 0.02, 0.05]
}
returns = pd.DataFrame(data)

weights = [0.5, 0.3, 0.2]

A portfolio object can now be created. For a set of portfolios where we want to optimize separately, we would create multiple portfolio objects.

In [5]:
#Create a portfolio object
portfolio = Portfolio(returns, weights)

You can see the optimized output below based on a maximized Sharpe ratio objective:

In [6]:
#Demonstrate optimization
optimized_result = portfolio.optimize(objective="sharpe")
print("Optimized Weights:", optimized_result.x)
print("Optimized Portfolio Return:", portfolio.get_return(optimized_result.x))
print("Optimized Portfolio Variance:", portfolio.get_var(optimized_result.x))


Optimized Weights: [0.70977761 0.         0.29022239]
Optimized Portfolio Return: 0.02572555597885054
Optimized Portfolio Variance: 0.0001565959607366926


Thank you for viewing this portfolio project!